# Mapping seismic stations in the Himalayas with Numpy and Matplotlib
## Or reading datasets with custom dtypes and plotting Earth-based data with basemap

In this exercise, we consider loading measurement files with the format:

<pre>
# Station  Lat    Long   Elev 
BIRA	26.4840	87.2670	0.0120
BUNG	27.8771	85.8909	1.1910
etc...
</pre>

These are seismic measurement stations in the Himalaya, with the elevation indicated in km.  Data with a structure such as this is common in many disciplines, and because we have a combination of text and numerical fields, we can't directly load it into a regular numpy array.

But we can use numpy's ability to [define custom data types (dtypes)](http://docs.scipy.org/doc/numpy/reference/arrays.dtypes.html) to compactly describe our data in a single array, which we can then manipulate.

If you have the basemap matplotlib toolkit installed, at the end of this example we will show a real Earth map and overlay the station locations on top of that.

We start by configuring pylab support and loading the required modules.

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

We want `numpy` to work with our file directly. To do this we can define a custom `dtype` as a list of four tuples. The first entry of the tuple should be a string, which you should take from the column headers in the file. The second entry of the tuple should describe the type of the measurement, which will be `'S4'` for the `Station` (a string of length 4), and `np.float32` for the other entries.

Construct this list, assigning it to `dt`.

In [ ]:
# Data descriptor to make a proper array.
# dt = ...

Now, we load the data using this dtype we've constructed, and view it as a `recarray` for convenient named-field access:

In [ ]:
data_fname = os.path.join('data', 'stations.txt')
tab = np.loadtxt(data_fname, dt).view(np.recarray)

You can now print out key information about the stations. In addition to the information below, print the mean latitude of the stations (which you should be able to do with one command).

In [ ]:
ptitle = 'Seismic stations in the Himalaya'
print(ptitle)
print('Stations:', tab.station)
print('Elevations (km):', tab.elev)
print('First station:', tab[0])

Now we can use `matplotlib` to plot the relative locations of the stations. Create such a plot using `plt.scatter` (the $x$ "coordinate" should be the longitude, and the $y$ "coordinate" the latitude). Improve the appearance of the plot by

1. Scaling the size of the points according to the elevation
2. Coloring the points according to the elevation.
3. Adding the "name" of the station as text to the plot, next to the point location.

In [ ]:
f1, ax = plt.subplots(figsize = (8,5))

# Make the size of the circles proportional to the elevation

# Add scatter plot

# The colorbar must be associated with the return value of scatter()
f1.colorbar(s)
ax.set_title(ptitle)

# Now add text labels for all the stations.  

# Note: when accessing single elements of the recarray, the named field
# syntax doesn't work and we must access the fields via ['name']
for record in tab:
    # Code here

To improve the plot further, we can use the `matplotlib` `basemap` toolkit. If you don't have it already installed and are using conda, then try

```bash
conda install basemap
```

Work through the code below to understand how the points are mapped onto the image and contours.

In [ ]:
try:
    from mpl_toolkits.basemap import Basemap
except ImportError:
    print("Basemap Not Available, no further plotting.")
else:
    # Draw the stations on a real map of the Earth.
    # Find boundaries 
    lon0 = 0.995*tab.lon.min()
    lon1 = 1.01*tab.lon.max()
    lat0 = 0.995*tab.lat.min()
    lat1 = 1.01*tab.lat.max()
    # Geographic grid to draw
    parallels = np.linspace(lat0, lat1, 5)
    meridians = np.linspace(lon0, lon1, 5)

    # Resolution of the basemap to load ('f' is *very* expensive)
    resolution = 'i' # intermediate resolution for map info

    f2, ax2 = plt.subplots(figsize=(10,6))
    m = Basemap(lon0, lat0, lon1, lat1, resolution=resolution, ax=ax2)
    m.drawcountries(color=(1,1,0))  # country boundaries yellow
    m.drawrivers(color=(0,1,1))  # rivers in cyan
    m.bluemarble()  # NASA bluemarble image
    m.drawparallels(parallels, labels=[1,0,0,0], fmt='%.2f')
    m.drawmeridians(meridians, labels=[0,0,0,1], fmt='%.2f')
    s = m.scatter(tab.lon, tab.lat, s=sizes, c=tab.elev, zorder=10, alpha=0.6)
    f2.colorbar(s)
    ax2.set_title(ptitle)
    for record in tab:
        ax2.text( record['lon']+0.05, record['lat']+0.05, record['station'], 
                weight='bold', color='yellow', zorder=10)